<a href="https://colab.research.google.com/github/praveensrikar/DiffusionLM/blob/main/DiffusionLM_E2E_Colab_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 Diffusion-LM on E2E NLG Dataset with Cosine Noise Schedule (Google Colab)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 📦 Step 1: Set Project Location





In [20]:
BASE_DIR= "/content/drive/MyDrive/Colab_Code/UA_MS_IS_ML_23-25/Capstone/05_Code/DiffusionLM"
%cd {BASE_DIR}

/content/drive/MyDrive/Colab_Code/UA_MS_IS_ML_23-25/Capstone/05_Code/DiffusionLM


### 📦 Step 1.1: (if Needed) Clone Diffusion-LM repository, install dependencies, Download the E2E dataset and Preprocess the Dataset

In [ ]:
# # Clone the repo
# !git clone https://github.com/XiangLi1999/Diffusion-LM.git
# %cd Diffusion-LM

# # Add repo to Python path (instead of pip install)
# import sys
# sys.path.append('/content/Diffusion-LM')


In [ ]:
# !mkdir -p data/e2e_data
# !wget https://github.com/tuetschek/e2e-dataset/blob/master/trainset.csv -O data/e2e_data/trainset.csv
# !wget https://github.com/tuetschek/e2e-dataset/blob/master/devset.csv -O data/e2e_data/devset.csv
# !wget https://github.com/tuetschek/e2e-dataset/blob/master/testset.csv -O data/e2e_data/testset.csv

### ✅ Step 1.2: Install Package on session

In [3]:
!pip install -q -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 466.3/466.3 kB 28.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 464.1/464.1 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 124.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 80.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 115.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 129.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━

In [4]:
!pip install benepar

  Using cached benepar-0.2.0-py3-none-any.whl
  Using cached torch_struct-0.5-py3-none-any.whl.metadata (4.3 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cach

In [24]:
!pip install mpi4py

  Using cached mpi4py-4.0.3-cp311-cp311-linux_x86_64.whl


# ✅ Step 2: Import all required libraries

In [27]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import transformers
import datasets
import spacy
import wandb
import benepar
import torch
import matplotlib.pyplot as plt
import re

print("✅ All packages imported successfully!")

torch.cuda.is_available()
if torch.cuda.is_available():
  device = torch.device("cuda")
  print(f'There are {torch.cuda.device_count()} GPU(s) available.')
  print('Device name:', torch.cuda.get_device_name(0))
  print('Device capability:', torch.cuda.get_device_capability(0))
  print('Device total memory [GB]:', torch.cuda.get_device_properties(0).total_memory / 1e9)
  !nvidia-smi
else:
  print('No GPU available, using the CPU instead.')
  device = torch.device("cpu")

✅ All packages imported successfully!
There are 1 GPU(s) available.
Device name: NVIDIA A100-SXM4-40GB
Device capability: (8, 0)
Device total memory [GB]: 42.474471424
Mon Jun  9 19:26:08 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   43C    P0             49W /  400W |       5MiB /  40960M

In [7]:
!pip install -e improved-diffusion/
!pip install -e transformers/

Obtaining file:///content/drive/MyDrive/Colab_Code/UA_MS_IS_ML_23-25/Capstone/05_Code/DiffusionLM/improved-diffusion
  Preparing metadata (setup.py) ... done
  Running setup.py develop for improved-diffusion
Obtaining file:///content/drive/MyDrive/Colab_Code/UA_MS_IS_ML_23-25/Capstone/05_Code/DiffusionLM/transformers
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 42.4 MB/s eta 0:00:00
  Building editable for transformers (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.17.0.dev0-0.editable-py3-none-any.whl size=24509 sha256=dc8824477250897e0a8601e8de241beb78b5e862d7edfda59034ee2133f484c6
  Stored in directory: /tmp/pip-ephem-wheel-cache-0ohrhgjc/wheels/a5/94/d8/5b4b332cfb8b1ddc7be4c15f0839360d49b5d8b397eb1e061f
Successfully built trans

## 📁 Step 4: Preprocess the Dataset
```python
## use this when imported ftom github
# !python improved-diffusion/scripts/preprocess_data.py \
#     --data_dir data/e2e_data \
#     --dataset datasets/e2e_data \
#     --tokenizer gpt2 \
#     --output_dir datasets/processed_e2e
```

In [5]:
!python improved-diffusion/scripts/preprocess_data.py \
    --data_dir data/e2e_data \
    --dataset datasets/e2e_data \
    --tokenizer gpt2 \
    --output_dir datasets/processed_e2e

98161
["David noticed he had put on a lot of weight recently. He examined his habits to try and figure out the reason. He realized he'd been eating too much fast food lately. He stopped going to burger places and started a vegetarian diet. After a few weeks, he started to feel much better.", "Tom had a very short temper. One day a guest made him very angry. He punched a hole in the wall of his house. Tom's guest became afraid and left quickly. Tom sat on his couch filled with regret about his actions."]

['The dog jumped up on the bed. I told her to go underneath it like she usually does. She refused to budge from her spot. I tried to make enough room for my legs. I gave up and let her sleep partly on top of me.', 'My family was excited for the fourth of july. We decided to have a Bar BQ. A lot of friends attended the event. We ate burgers. At the end of the night we watched fireworks.']


# Train the Model (Default Noise Schedule)

```python
# !python improved-diffusion/scripts/train.py \
#     --dataset datasets/e2e_data \
#     --data_dir datasets/processed_e2e \
#     --output_dir datasets/trained_e2e_baseline \
#     --num_train_epochs 5 \
#     --learning_rate 5e-5
```


In [28]:
%cd /content/drive/MyDrive/Colab_Shared/Diffusion-LM/improved-diffusion
!python scripts/run_train.py --diff_steps 2000 --model_arch transformer --lr 0.0001 --lr_anneal_steps 200000  --seed 102 --noise_schedule sqrt --in_channel 16 --modality e2e-tgt --submit no --padding_mode block --app "--predict_xstart True --training_mode e2e --vocab_size 821  --e2e_train ../datasets/e2e_data " --notes xstart_e2e --bsz 128 > decode_log.txt

/content/drive/MyDrive/Colab_Shared/Diffusion-LM/improved-diffusion
2025-06-09 19:26:38.718940: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-09 19:26:38.736925: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749497198.758469   15561 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749497198.765094   15561 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-09 19:26:38.786639: I tensorflow/core/platform/cpu_feature_guard.cc

In [ ]:
#backup !python improved-diffusion/scripts/run_train.py \
#       --diff_steps 2000 \
#       --model_arch transformer \
#       --lr 0.0001 \
#       --lr_anneal_steps 200000  \
#       --seed 102 \
#       --noise_schedule sqrt \
#       --in_channel 16 \
#       --modality e2e-tgt \
#       --submit no \
#       --padding_mode block \
#       --notes xstart_e2e

In [ ]:
# !mkdir -p generation_outputs
# %cd "/content/drive/MyDrive/Colab_Shared/Diffusion-LM"

In [ ]:


log_file_path = 'improved-diffusion/diffusion_models/diff_e2e-tgt_block_rand16_transformer_lr0.0001_0.0_2000_sqrt_Lsimple_h128_s2_d0.1_sd102_xstart_e2e/log.txt'

steps = []
losses = []

# Regex to find lines containing step and loss
step_loss_pattern = re.compile(r"\|\sstep\s+\|\s(\d+)\s+\|.*\|\sloss\s+\|\s([\d\.]+)\s+\|", re.DOTALL)

with open(log_file_path, 'r') as f:
    content = f.read()

    # Find all matches for the pattern
    matches = step_loss_pattern.findall(content)

    for match in matches:
        # The first training step log is usually a summary of eval loss,
        # we will capture the detailed training steps that follow.
        pass # The initial block is structured differently.

# A more robust way to capture the loss is to look for the specific table format.
log_blocks = content.split('------------------------')
for block in log_blocks:
    if 'loss' in block and 'step' in block:
        try:
            step_search = re.search(r"\|\s*step\s*\|\s*([\d.e+]+)\s*\|", block)
            loss_search = re.search(r"\|\s*loss\s*\|\s*([\d.]+)\s*\|", block)

            if step_search and loss_search:
                step = int(float(step_search.group(1)))
                loss = float(loss_search.group(1))
                steps.append(step)
                losses.append(loss)
        except (ValueError, IndexError) as e:
            print(f"Skipping block due to parsing error: {e}\n{block}")


# Plotting the graph
plt.figure(figsize=(12, 6))
plt.plot(steps, losses, marker='o', linestyle='-')
plt.title('Training Loss vs. Steps for Sqrt Noise Schedule')
plt.xlabel('Training Steps')
plt.ylabel('Loss')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Define the model path pattern as a shell variable
MODEL_PATH_PATTERN="improved-diffusion/diffusion_models/diff_e2e-tgt_block_rand16_transformer_lr0.0001_0.0_2000_sqrt_Lsimple_h128_s2_d0.1_sd102_xstart_e2e*"

# Run the batch_decode.py script with the corrected pattern
!python /content/drive/MyDrive/Colab_Shared/Diffusion-LM/improved-diffusion/scripts/batch_decode.py "$MODEL_PATH_PATTERN" 0.95 "ema_0.9999_050000"

In [ ]:
benepar.download('benepar_en3')

In [ ]:
!python improved-diffusion/control_gen/eval_control.py \
--input_text generation_outputs/diff_e2e-tgt_block_rand16_transformer_lr0.0001_0.0_2000_sqrt_Lsimple_h128_s2_d0.1_sd102_xstart_e2e.ema_0.9999_200000.pt.samples_-1.0.json \
--input_format paired \
--mode e2e-tgt-ppl \
--model_name_or_path predictability/diff_models/e2e-tgt_e=20_b=64_m=gpt2_wikitext-103-raw-v1_101_wp_finetune_UNK

## 🛠️ Step 5: Modify noise schedule to cosine

You can modify the noise schedule inside the `schedulers/diffusion_schedule.py` file. Replace the `linear_beta_schedule` with a `cosine_beta_schedule`:

```python
import numpy as np

def cosine_beta_schedule(timesteps, s=0.008):
    steps = timesteps + 1
    x = np.linspace(0, timesteps, steps)
    alphas_cumprod = np.cos(((x / timesteps) + s) / (1 + s) * np.pi * 0.5) ** 2
    alphas_cumprod = alphas_cumprod / alphas_cumprod[0]
    betas = 1 - (alphas_cumprod[1:] / alphas_cumprod[:-1])
    return np.clip(betas, 0.0001, 0.9999)
```


## 🧪 Step 6: Training & Inference
Follow instructions in the Diffusion-LM `README.md` to launch training.